In [1]:
import collections
from tkinter import filedialog
import matplotlib as mpl 
import matplotlib.pyplot as plt 
from config_radar import CustomConfig
from processing_chain import ProcessingChain
from data_handling import DataHandling
from mmwave import dsp
import scipy.io as sio  
import numpy as np 
import glob, os
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline
norm = mpl.colors.Normalize(vmin=0.,vmax=1.)

In [2]:
# global variables
num_tx = 1
num_rx = 4
res_range = 0.05
max_range = 5.75
max_velocity = 5
fps = 20
fs = 3200

In [3]:
def config_1843():
    # (1) configure radar and print resultant radarcube parameters

    # (1.2) instantiate radar and run config script 
    awr1843_config = CustomConfig(num_tx=num_tx, num_rx=num_rx, res_range=res_range, max_range=max_range, max_velocity=max_velocity, fps=fps, fs=fs)    
    config_params_1843 = awr1843_config._run_config()
    print("\nIf sensorStart status is not 'Done', something could be wrong in configuring the radar! \n")

    # (1.3) print resultant rcube parameters
    print('{:-^60}'.format(' resultant rcube parameters '))
    for key, value in config_params_1843.items():
        print('{:25}'.format(key + ':') + str(round(value,4)))
    print('{:-^60}'.format(''))

def organize_rawdata(num_samples, num_chirps, experiment_name):
    # (2) organize raw data 

    # (2.2) instantiate DataHandling class
    data_handle = DataHandling(num_samples=num_samples, num_chirps=num_chirps, num_tx=num_tx, num_rx=num_rx, fps=fps)
    # (2.3) organize data if new data is present
    dir_name = data_handle.organize_captured_data(experiment_name=experiment_name)

    return dir_name

def perform_rangedoppler_processing(dir_name, log_scaled=False, process_single_datafile=True, window=3, interp_factor=1):
  
    folder_name = dir_name.split("/")[-1]
    #print(folder_name)
    params = folder_name.split('_')

    num_samples = int(params[4].split('x')[0][1:])
    num_chirps = int(params[4].split('x')[1][1:])
    num_tx = int(params[3][3])
    num_rx = int(params[3][4])
    fps = int(params[5][3:])

    data_process = ProcessingChain(num_samples=num_samples, num_chirps=num_chirps, 
                num_tx=num_tx, num_rx=num_rx, fps=fps, window=window, interp_factor=interp_factor)
    radarcube =  data_process.range_doppler_process(dir_name, log_scaled=log_scaled, 
                process_single_datafile=process_single_datafile, normalize=False)
    return radarcube, folder_name

def perform_microdoppler_processing(dir_name, interp_factor=1, window=3, accum_type=0):
    
    folder_name = dir_name.split("/")[-1]
    print(folder_name)
    params = folder_name.split('_')
    #print('params: ' + str(params))
    num_samples = int(params[4].split('x')[0][1:])
    num_chirps = int(params[4].split('x')[1][1:])
    num_tx = int(params[3][3])
    num_rx = int(params[3][4])
    fps = int(params[5][3:])

    data_process = ProcessingChain(num_samples=num_samples, num_chirps=num_chirps,             
                    num_tx=num_tx, num_rx=num_rx, fps=fps, window=window, interp_factor=interp_factor)
    micro_doppler_spectrum = data_process.micro_doppler_stft(dir_name, max_velocity, accum_type=accum_type)
    return micro_doppler_spectrum, folder_name

In [ ]:
#--------------------------#
# MICRO-DOPPLER PROCESSING
#--------------------------#
#dir_name = "/mnt/c/work/mmw_pc/single_chip/datasets/may20_slow_moving_test/dca_may20_1743_trx14_n128xp128_fps20_slow_walk_back_forth"
dir_name =    filedialog.askdirectory(initialdir='/mnt/c/work/mmw_pc/single_chip/datasets')
#print(dir_name)
savemat = False
multiple_files = True
window = 2
accum_type = 0
dir = []
if multiple_files:
    # load the files in directory and loop over them to generate all micro-Doppler spectrograms 
    for subdir in os.walk(dir_name):
        dir.append(subdir[0])
else:
    dir.append(dir_name)
    dir.append(dir_name)

for i in range(1, len(dir)):
    plt.figure()
    #for accum_type in range(2):
    #for window in range(1,7):
    if True:
        micro_doppler_spectrum, _ = perform_microdoppler_processing(dir[i], 
                                    window=window, accum_type=accum_type)
        if savemat:
            md_dict = {}
            md_dict[dir[i].split('/')[-1]] = micro_doppler_spectrum
            sio.savemat(dir[i] + '.mat', md_dict, appendmat=True)
    
        #plt.subplot(3,2, window)
        #plt.subplot(2,1,accum_type+1)
        ax = plt.gca()
        im = ax.imshow(micro_doppler_spectrum, extent=[0, micro_doppler_spectrum.shape[1]/fps, 
             -max_velocity, max_velocity], cmap='gnuplot2')
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="2%", pad=0.02)
        plt.colorbar(im, cax=cax)

    plt.savefig('plots/accum_type/hamming_bart/' + dir[i].split('/')[-1] + '.jpg', 
                dpi=1000, bbox_inches = 'tight', pad_inches = 0.1)

print('Finished micro-Doppler processing!\n')


In [4]:
#--------------------------#
#     RANGE PROCESSING
#--------------------------#

#dir_name = "/mnt/c/work/mmw_pc/single_chip/datasets/may15_slow_moving_test/dca_may15_1800_trx14_n128xp128_fps20_slow_walk_front_and_off"
dir_name =    filedialog.askdirectory(initialdir='/mnt/c/work/mmw_pc/single_chip/datasets')
print(dir_name)
multiple_files = False
process_single_datafile = False 
frames = [300, 310, 320, 330]
save_rcube = True

window = 3
dir = []
if multiple_files:
    # load the files in directory and loop over them to generate all micro-Doppler spectrograms 
    for subdir in os.walk(dir_name):
        dir.append(subdir[0])
else:
    dir.append(dir_name)
    dir.append(dir_name)

for i in range(1, len(dir)):
    plt.figure()
    #for accum_type in range(3):
    #for window in range(1,7):
    if True: 
        radarcube, folder_name = perform_rangedoppler_processing(log_scaled=True, 
                                 process_single_datafile=process_single_datafile, dir_name=dir[i], window=window)

        if save_rcube:
            full_path = 'rcube_3d/' + folder_name + '.bin'
            print(full_path)
            with open(full_path, 'wb') as f:
                np.save(f, radarcube)

            with open(full_path, 'rb') as f:
                rcube = np.load(f)
            rcube.shape
    
#for frame in frames:
#    plt.imshow(radarcube[:, :, frame], cmap='gnuplot2')
#    plt.show()


/mnt/c/work/mmw_pc/single_chip/datasets/jun11_rres_effect_experiment_pot_swing/dca_jun11_1704_trx14_n128xp128_fps20_rres_10cm_pot_swing_2
range-doppler video written successfully ...

rcube_3d/dca_jun11_1704_trx14_n128xp128_fps20_rres_10cm_pot_swing_2.bin


<Figure size 432x288 with 0 Axes>

In [ ]:
#--------------------------#
# RANGE-DOPPLER PROCESSING
#--------------------------#

dir_name =    filedialog.askdirectory(initialdir='/mnt/c/work/mmw_pc/single_chip/datasets')
dir = []
for subdir in os.walk(dir_name):
    dir.append(subdir[0])

for i in range(1, len(dir)):
    single_file = False
    radarcube, folder_name = perform_rangedoppler_processing(log_scaled=True, dir_name=dir[i], single_file=single_file, window=3)

    if save_rcube:
        full_path = 'rcube_3d/' + folder_name + '_hann' + '.bin'
        print(full_path)
        with open(full_path, 'wb') as f:
            np.save(f, radarcube)

        with open(full_path, 'rb') as f:
            rcube = np.load(f)
        rcube.shape